In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('data2.csv')
df.head(7)

,artist_name,id,track_name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,artist_pop,genres,track_pop,genres_list,subjectivity,polarity
0,Missy Elliott,0UaMYEvWZi0ZqiDOoHU3YI,Lose Control (feat. Ciara & Fat Man Scoop),0.904,0.813,4,-7.105,0,0.1210,0.03110,0.006970,0.0471,0.810,125.461,74,dance_pop hip_hop hip_pop pop pop_rap r&b rap ...,69,"['dance_pop', 'hip_hop', 'hip_pop', 'pop', 'po...",low,Neutral
1,Britney Spears,6I9VzXrHxO9rA9A5euc8Ak,Toxic,0.774,0.838,5,-3.914,0,0.1140,0.02490,0.025000,0.2420,0.924,143.040,84,dance_pop pop post-teen_pop,83,"['dance_pop', 'pop', 'post-teen_pop']",low,Neutral
2,Beyoncé,0WqIKmW4BTrj3eJFmnCKMv,Crazy In Love,0.664,0.758,2,-6.583,0,0.2100,0.00238,0.000000,0.0598,0.701,99.259,86,dance_pop pop r&b,25,"['dance_pop', 'pop', 'r&b']",high,Negative
3,Justin Timberlake,1AWQoqb9bSvzTjaLralEkT,Rock Your Body,0.892,0.714,4,-6.055,0,0.1410,0.20100,0.000234,0.0521,0.817,100.972,82,dance_pop pop,79,"['dance_pop', 'pop']",low,Neutral
4,Shaggy,1lzr43nnXAijIGYnCT8M8H,It Wasn't Me,0.853,0.606,0,-4.596,1,0.0713,0.05610,0.000000,0.3130,0.654,94.759,75,pop_rap reggae_fusion,2,"['pop_rap', 'reggae_fusion']",low,Neutral
5,Usher,0XUfyU2QviPAs6bxSpXYG4,Yeah!,0.881,0.788,2,-4.669,1,0.1680,0.02120,0.000000,0.0377,0.592,104.997,82,atl_hip_hop contemporary_r&b dance_pop pop r&b...,0,"['atl_hip_hop', 'contemporary_r&b', 'dance_pop...",low,Neutral
6,Usher,68vgtRHr7iZHpzGpon6Jlo,My Boo,0.662,0.507,5,-8.238,1,0.1180,0.25700,0.000000,0.0465,0.676,86.412,82,atl_hip_hop contemporary_r&b dance_pop pop r&b...,79,"['atl_hip_hop', 'contemporary_r&b', 'dance_pop...",low,Neutral


In [3]:
df.drop(['id', 'track_name', 'genres_list'], axis=1, inplace=True)

In [4]:
df["artist_name"] = (df["artist_name"].str.strip()).str.lower()
df["genres"] = (df["genres"].str.strip()).str.lower()

In [5]:
columns = ['artist_name', 'subjectivity', 'polarity']
le = LabelEncoder()
for col in columns:
    df[col] = le.fit_transform(df[col])

In [6]:
df["loudness"] = df["loudness"] + 60

In [7]:
columns = ['artist_name', 'key', 'artist_pop', 'track_pop', 'subjectivity', 'polarity']
for col in columns:
    df[col] = df[col] / df[col].max()

In [8]:
columns = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
for col in columns:
    df[col] = df[col] / df[col].max()

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34247 entries, 0 to 34246
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   artist_name       34247 non-null  float64
 1   danceability      34247 non-null  float64
 2   energy            34247 non-null  float64
 3   key               34247 non-null  float64
 4   loudness          34247 non-null  float64
 5   mode              34247 non-null  int64  
 6   speechiness       34247 non-null  float64
 7   acousticness      34247 non-null  float64
 8   instrumentalness  34247 non-null  float64
 9   liveness          34247 non-null  float64
 10  valence           34247 non-null  float64
 11  tempo             34247 non-null  float64
 12  artist_pop        34247 non-null  float64
 13  genres            34247 non-null  object 
 14  track_pop         34247 non-null  float64
 15  subjectivity      34247 non-null  float64
 16  polarity          34247 non-null  float6

In [10]:
df.describe()

,artist_name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,artist_pop,track_pop,subjectivity,polarity
count,34247.000000,34247.000000,34247.000000,34247.000000,34247.000000,34247.000000,34247.000000,34247.000000,34247.000000,34247.000000,34247.000000,34247.000000,34247.000000,34247.000000,34247.000000,34247.000000
mean,0.498793,0.591795,0.636613,0.474019,0.834741,0.665080,0.094766,0.264799,0.077573,0.197244,0.486990,0.554970,0.619166,0.320773,0.395728,0.536908
std,0.288269,0.166302,0.225477,0.325780,0.063206,0.471969,0.105089,0.301755,0.219825,0.166794,0.244118,0.133595,0.191201,0.262561,0.208142,0.245965
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.251724,0.481781,0.490000,0.181818,0.809905,0.000000,0.036486,0.020181,0.000000,0.095800,0.292585,0.449723,0.510000,0.000000,0.500000,0.500000
50%,0.496964,0.599190,0.669000,0.454545,0.849266,1.000000,0.050832,0.126506,0.000008,0.128000,0.477956,0.551444,0.650000,0.350515,0.500000,0.500000
75%,0.741433,0.713563,0.818000,0.727273,0.876239,1.000000,0.099584,0.446787,0.002714,0.252000,0.678357,0.638550,0.760000,0.546392,0.500000,0.500000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [11]:
# genre_list = []
# for genres in df['genres']:
#     genres = genres.split(' ')
#     for genre in genres:
#         genre_list.append(genre)

In [12]:
# cnt = 0
# for i, genre in enumerate(genre_list):
#     if 'trap' in genre:
#         cnt += 1
# cnt

In [13]:
main_genres = ['contemporary', 'country', 'dance', 'folk', 'hip_hop',
               'house', 'indie', 'metal', 'modern', 'pop',
               'rap', 'rock', 'r&b', 'soul', 'trap'
               ]

In [14]:
genre_to_class = { 'other' : 0,
                   'contemporary' : 1,
                   'country' : 2,
                   'dance' : 3,
                   'folk' : 4,
                   'hip_hop' : 5,
                   'house' : 6,
                   'indie' : 7,
                   'metal' : 8,
                   'modern' : 9,
                   'pop' : 10,
                   'rap' : 11,
                   'rock' : 12,
                   'r&b' : 13,
                   'soul' : 14,
                   'trap' : 15
}

In [15]:
class_to_genre = { 0 : 'other',
                   1 : 'contemporary',
                   2 : 'country',
                   3 : 'dance',
                   4 : 'folk',
                   5 : 'hip_hop',
                   6 : 'house',
                   7 : 'indie',
                   8 : 'metal',
                   9 : 'modern',
                   10 : 'pop',
                   11 : 'rap',
                   12 : 'rock',
                   13 : 'r&b',
                   14 : 'soul',
                   15 : 'trap',
}

In [16]:
genre_classes = np.zeros((df.shape[0], len(genre_to_class)))
for i, genres in enumerate(df['genres']):
    genres = genres.split(' ')
    for genre in genres:
        other = True
        for main_genre in main_genres:
            if main_genre in genre:
                genre_classes[i, genre_to_class[main_genre]] = 1
                other = False
        if other:
            genre_classes[i, genre_to_class['other']] = 1

In [17]:
df.drop(['genres'], axis=1, inplace=True)
number_features = 16

In [18]:
df_train,df_test,y_train,y_test = train_test_split(df,genre_classes, test_size=0.15, random_state=12)

In [19]:
x_train = df_train.to_numpy()
x_test = df_test.to_numpy()

In [20]:
x_train,x_val,y_train,y_val = train_test_split(x_train,y_train, test_size=0.1765, random_state=12)

In [21]:
class spotifyDataset(Dataset):
    def __init__(self, x, y):
        super().__init__()
        self.x = x
        self.y = y

    def __len__(self):
        return self.x.shape[0]

    def __getitem__(self, idx):
        x = self.x[idx, :]
        y = self.y[idx, :]
        return x,y

In [22]:
dataset_train = spotifyDataset(x_train, y_train)
dataset_val = spotifyDataset(x_val, y_val)
dataset_test = spotifyDataset(x_test, y_test)

In [23]:
train_loader = DataLoader(dataset_train, batch_size = 1024, shuffle = True)
val_loader = DataLoader(dataset_val, batch_size = 1, shuffle = False)
test_loader = DataLoader(dataset_test, batch_size = 1, shuffle = False)

In [24]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.bn0 = nn.BatchNorm1d(number_features)

        self.fc1 = nn.Linear(number_features, 128)
        self.bn1 = nn.BatchNorm1d(128)
        self.drop1 = nn.Dropout(p=0.1)

        self.fc2 = nn.Linear(128, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.drop2 = nn.Dropout(p=0.1)

        # self.fc3 = nn.Linear(64, 32)
        # self.bn3 = nn.BatchNorm1d(32)
        # self.drop3 = nn.Dropout(p=0.1)

        self.leakyrelU = nn.LeakyReLU()
        self.out = nn.Linear(64, len(genre_to_class))
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.bn0(x)

        x = self.fc1(x)
        x = self.bn1(x)
        x = self.leakyrelU(x)
        x = self.drop1(x)

        x = self.fc2(x)
        x = self.bn2(x)
        x = self.leakyrelU(x)
        x = self.drop2(x)

        # x = self.fc3(x)
        # x = self.bn3(x)
        # x = self.leakyrelU(x)
        # x = self.drop3(x)

        x = self.out(x)
        x = self.sigmoid(x)
        return x

In [25]:
model = MLP()
model = model.float()

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = StepLR(optimizer, step_size=5, gamma=0.8)

In [26]:
model_name = './model.pt'

In [28]:
model.load_state_dict(torch.load(model_name))
model.eval()
preds = None
targets = None
val_loss_best = 0
for i, (data, target) in enumerate(test_loader):
    output = model(data.float())
    pred = output.detach().numpy()
    loss = criterion(output, target.float())
    val_loss_best += loss.item()*data.size(0)

    if i==0:
        preds = pred
        targets = target.detach().cpu().numpy()
    else:
        preds = np.concatenate((preds, pred), axis=0)
        targets = np.concatenate((targets, target.detach().cpu().numpy()), axis=0)

val_loss_best = val_loss_best / len(test_loader.dataset)
val_loss_best

0.27411006400185156

In [34]:
prediction_limit = 2

In [35]:
CM = np.zeros((len(genre_to_class)+1, len(genre_to_class)+1))
for k in range(preds.shape[0]):
    indices = np.argsort(preds[k])
    for i in range(len(genre_to_class)-1, -1, -1):
        l = indices[i]
        if i+prediction_limit >= len(genre_to_class):
            if preds[k,l] >= 0.5:
                if targets[k,l] == 1:
                    CM[l,l] += 1
                else:
                    CM[len(genre_to_class),l] += 1
            else:
                if targets[k,l] == 1:
                    CM[l,len(genre_to_class)] += 1

In [36]:
np.savetxt('confusion_matrix_limited.csv', CM, delimiter=',')

In [37]:
100 * np.trace(CM) / np.sum(CM)

66.33809200090641

In [38]:
hit_count = 0
for k in range(preds.shape[0]):
    indices = np.argsort(preds[k])
    for i in range(len(genre_to_class)-1, -1, -1):
        l = indices[i]
        if i+prediction_limit >= len(genre_to_class):
            if preds[k,l] >= 0.5:
                if targets[k,l] == 1:
                    hit_count += 1
                    break
100*hit_count/preds.shape[0]

82.81432463993772

In [39]:
CMs = []
for l in range(len(genre_to_class)):
    CMs.append(np.zeros((2,2)))

In [40]:
for k in range(preds.shape[0]):
    indices = np.argsort(preds[k])
    for i in range(len(genre_to_class)-1, -1, -1):
        l = indices[i]
        if i+prediction_limit >= len(genre_to_class):
            if preds[k,l] >= 0.5:
                if targets[k,l] == 1:
                    CMs[l][1,1] += 1
                else:
                    CMs[l][0,1] += 1
            else:
                if targets[k,l] == 1:
                    CMs[l][1,0] += 1
                else:
                    CMs[l][0,0] += 1

In [41]:
accuracies = []
counts = []
for i in range(len(genre_to_class)):
    # print(CMs[i])
    counts.append(np.sum(CMs[i]))
    accuracies.append(100 * np.trace(CMs[i]) / np.sum(CMs[i]))
    print(accuracies[-1])
np.dot(counts, accuracies) / np.sum(counts)

72.28485015122354
75.0
67.8082191780822
66.43835616438356
80.23255813953489
75.32956685499059
76.36363636363636
76.36363636363636
66.33663366336634
90.9090909090909
67.1280276816609
75.05827505827506
69.96779388083736
58.53658536585366
69.44444444444444
83.33333333333333


71.08797197353056

In [43]:
prediction_limit = 2

In [44]:
CM = np.zeros((len(genre_to_class)+1, len(genre_to_class)+1))
for k in range(preds.shape[0]):
    indices = np.argsort(preds[k])
    hit = False
    for i in range(len(genre_to_class)-1, -1, -1):
        l = indices[i]
        if i+prediction_limit >= len(genre_to_class):
            if preds[k,l] >= 0.5:
                if targets[k,l] == 1:
                    CM[l,l] += 1
                    hit = True
                else:
                    CM[len(genre_to_class),l] += 1
    if not hit:
        for i in range(len(genre_to_class)-1, -1, -1):
            l = indices[i]
            if i+prediction_limit >= len(genre_to_class):
                if preds[k,l] < 0.5:
                    if targets[k,l] == 1:
                        CM[l,len(genre_to_class)] += 1

In [45]:
np.savetxt('confusion_matrix_limited_stricted.csv', CM, delimiter=',')

In [46]:
100 * np.trace(CM) / np.sum(CM)

69.13449049474555

In [47]:
hit_count = 0
for k in range(preds.shape[0]):
    indices = np.argsort(preds[k])
    for i in range(len(genre_to_class)-1, -1, -1):
        l = indices[i]
        if i+prediction_limit >= len(genre_to_class):
            if preds[k,l] >= 0.5:
                if targets[k,l] == 1:
                    hit_count += 1
                    break
100*hit_count/preds.shape[0]

82.81432463993772

In [48]:
CMs = []
for l in range(len(genre_to_class)):
    CMs.append(np.zeros((2,2)))

In [49]:
for k in range(preds.shape[0]):
    indices = np.argsort(preds[k])

    hit = False
    for i in range(len(genre_to_class)-1, -1, -1):
        l = indices[i]
        if i+prediction_limit >= len(genre_to_class):
            if preds[k,l] >= 0.5:
                if targets[k,l] == 1:
                    CMs[l][1,1] += 1
                    hit = True
                else:
                    CMs[l][0,1] += 1
    if not hit:
        for i in range(len(genre_to_class)-1, -1, -1):
            l = indices[i]
            if i+prediction_limit >= len(genre_to_class):
                if preds[k,l] < 0.5:
                    if targets[k,l] == 1:
                        CMs[l][1,0] += 1
                    else:
                        CMs[l][0,0] += 1

In [50]:
accuracies = []
counts = []
for i in range(len(genre_to_class)):
    # print(CMs[i])
    counts.append(np.sum(CMs[i]))
    accuracies.append(100 * np.trace(CMs[i]) / np.sum(CMs[i]))
    print(accuracies[-1])
np.dot(counts, accuracies) / np.sum(counts)

73.0
71.71717171717172
67.4074074074074
65.47085201793722
64.0
74.89711934156378
70.96774193548387
62.27272727272727
65.21739130434783
66.66666666666667
64.57725947521865
75.09627727856225
70.31963470319634
61.29032258064516
62.5
90.9090909090909


70.28194633924511

In [51]:
cnt = 1

In [52]:
i = np.random.permutation(preds.shape[0])
xs = preds[i[0:cnt], :]
ys = targets[i[0:cnt], :]

In [53]:
for x,y in zip(xs,ys):
    print('---LABELS---')
    for a in range(len(genre_to_class)):
        if y[a] == 1:
            print(class_to_genre[a])

    print('---PREDS---')
    indices = np.argsort(x)
    for i in range(len(genre_to_class)-1, -1, -1):
        l = indices[i]
        if i+prediction_limit >= len(genre_to_class):
            if x[l] >= 0.5:
                print(class_to_genre[l])
    print()

---LABELS---
metal
rock
---PREDS---
rock
other

